In [174]:
import pandas as pd
import numpy as np
from sklearn import linear_model

## Dashboard 1
### Dataset 1 (Bus Demand)
url: https://datamall.lta.gov.sg/content/datamall/en/dynamic-data.html

Used: Passenger Volume By Bus Stops (from Oct22-Dec22)

In [175]:
df1 = pd.read_csv('./datasrc/transport_node_bus_202210.csv')
df2 = pd.read_csv('./datasrc/transport_node_bus_202211.csv')
df3 = pd.read_csv('./datasrc/transport_node_bus_202212.csv')

In [176]:
print(f"{'10-2022':*^28}")
print(df1.isnull().sum())
print(f"{'11-2022':*^28}")
print(df2.isnull().sum())
print(f"{'12-2022':*^28}")
print(df3.isnull().sum())


**********10-2022***********
YEAR_MONTH              0
DAY_TYPE                0
TIME_PER_HOUR           0
PT_TYPE                 0
PT_CODE                 0
TOTAL_TAP_IN_VOLUME     0
TOTAL_TAP_OUT_VOLUME    0
dtype: int64
**********11-2022***********
YEAR_MONTH              0
DAY_TYPE                0
TIME_PER_HOUR           0
PT_TYPE                 0
PT_CODE                 0
TOTAL_TAP_IN_VOLUME     0
TOTAL_TAP_OUT_VOLUME    0
dtype: int64
**********12-2022***********
YEAR_MONTH              0
DAY_TYPE                0
TIME_PER_HOUR           0
PT_TYPE                 0
PT_CODE                 0
TOTAL_TAP_IN_VOLUME     0
TOTAL_TAP_OUT_VOLUME    0
dtype: int64


In [177]:
# Merge all dataframes into one (row-wise)
df = pd.concat([df1, df2, df3], ignore_index=True, axis=0)
print(df.isnull().sum())

YEAR_MONTH              0
DAY_TYPE                0
TIME_PER_HOUR           0
PT_TYPE                 0
PT_CODE                 0
TOTAL_TAP_IN_VOLUME     0
TOTAL_TAP_OUT_VOLUME    0
dtype: int64


In [178]:
# Get rid of the outliers for TOTAL_TAP_IN_VOLUME and TOTAL_TAP_OUT_VOLUME
from scipy import stats
df = df[(np.abs(stats.zscore(df['TOTAL_TAP_IN_VOLUME'])) < 3)]
df = df[(np.abs(stats.zscore(df['TOTAL_TAP_OUT_VOLUME'])) < 3)]
df.describe()

,TIME_PER_HOUR,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
count,575185.000000,575185.000000,575185.000000,575185.000000
mean,13.279747,48941.690018,285.352535,267.458927
std,6.158887,25457.164274,486.533475,401.018905
min,0.000000,1012.000000,0.000000,0.000000
25%,9.000000,25421.000000,20.000000,22.000000
50%,14.000000,50038.000000,99.000000,106.000000
75%,18.000000,67141.000000,328.000000,330.000000
max,23.000000,99189.000000,4550.000000,2518.000000


In [179]:
# Get the mean across the three months to get monthly tap in and tap out
df = df.groupby(["PT_CODE","DAY_TYPE","TIME_PER_HOUR"]).mean().round().astype("int").sort_index()
df.rename(columns={"TOTAL_TAP_IN_VOLUME":"Monthly_TapIn","TOTAL_TAP_OUT_VOLUME":"Monthly_TapOut"}, inplace=True)
df.index.rename(['BusStopCode','DayType','TimePerHour'], inplace=True)
display(df)

Monthly_TapIn  Monthly_TapOut
BusStopCode DayType          TimePerHour                               
1012        WEEKDAY          0                      272              74
                             1                        3               1
                             5                       20              44
                             6                      190             303
                             7                      354             445
...                                                 ...             ...
99189       WEEKENDS/HOLIDAY 19                      65              97
                             20                      48              91
                             21                      38              56
                             22                      15              68
                             23                      12              27

[196614 rows x 2 columns]

In [180]:
print(df.dtypes)
print(df.isnull().sum())

Monthly_TapIn     int32
Monthly_TapOut    int32
dtype: object
Monthly_TapIn     0
Monthly_TapOut    0
dtype: int64


In [181]:
display(df.head(),df.tail())

Monthly_TapIn  Monthly_TapOut
BusStopCode DayType TimePerHour                               
1012        WEEKDAY 0                      272              74
                    1                        3               1
                    5                       20              44
                    6                      190             303
                    7                      354             445

Monthly_TapIn  Monthly_TapOut
BusStopCode DayType          TimePerHour                               
99189       WEEKENDS/HOLIDAY 19                      65              97
                             20                      48              91
                             21                      38              56
                             22                      15              68
                             23                      12              27

In [182]:
# Right back to csv (higher level of granularity)
df.to_csv('./data/bus_demand_high_LOD.csv')

In [183]:
# Downsample TimePerHour into four categories (6-8:59,9-16:59,17-18:59,19-5:59) so that it can merge with the categorized
# level of data in the bus
df.reset_index(inplace=True)
# Prepare data for pd.cut
beforeValueCount = df["TimePerHour"].value_counts().sort_index()
df["TimePerHour"] = df["TimePerHour"].apply(lambda x: x-6 if x>5 else x+18)
afterValueCount = df["TimePerHour"].value_counts().sort_index()

# Show the two series side by side
print(pd.concat([beforeValueCount,afterValueCount], axis=1, keys=["Before","After"]))

    Before  After
0     8318   9833
1     3936   9758
2      825   9690
3      327   9830
4      143   9854
5     7094   9874
6     9833   9881
7     9758   9891
8     9690   9908
9     9830   9884
10    9854   9840
11    9874   9710
12    9881   9589
13    9891   9706
14    9908   9823
15    9884   9669
16    9840   9647
17    9710   9584
18    9589   8318
19    9706   3936
20    9823    825
21    9669    327
22    9647    143
23    9584   7094


In [184]:
# Now we can use cut
# Bins 0,2 (6-8:59) ,10 (9-16:59), 12 (17-18:59), inf (After 19:00)
df["TimeCat"] = pd.cut(df["TimePerHour"], bins=[0,2,10,12,len(df["TimePerHour"])] , labels=["AM_Peak","AM_Offpeak","PM_Peak","PM_Offpeak"], right=True,include_lowest=True)
# Convert TimePerHour back
df["TimePerHour"] = df["TimePerHour"].apply(lambda x: x+6 if x<18 else x-18)
# Check in all unique rows whether Category is corretly assigned
display(df[["TimePerHour","TimeCat"]].drop_duplicates(keep='first').sort_values("TimePerHour"))

,TimePerHour,TimeCat
0,0,PM_Offpeak
1,1,PM_Offpeak
245,2,PM_Offpeak
3942,3,PM_Offpeak
4676,4,PM_Offpeak
2,5,PM_Offpeak
3,6,AM_Peak
4,7,AM_Peak
5,8,AM_Peak
6,9,AM_Offpeak


In [185]:
print(df.isnull().sum())

BusStopCode       0
DayType           0
TimePerHour       0
Monthly_TapIn     0
Monthly_TapOut    0
TimeCat           0
dtype: int64


In [186]:
display(df[df["BusStopCode"]==1012])

,BusStopCode,DayType,TimePerHour,Monthly_TapIn,Monthly_TapOut,TimeCat
0,1012,WEEKDAY,0,272,74,PM_Offpeak
1,1012,WEEKDAY,1,3,1,PM_Offpeak
2,1012,WEEKDAY,5,20,44,PM_Offpeak
3,1012,WEEKDAY,6,190,303,AM_Peak
4,1012,WEEKDAY,7,354,445,AM_Peak
5,1012,WEEKDAY,8,425,1029,AM_Peak
6,1012,WEEKDAY,9,356,1138,AM_Offpeak
7,1012,WEEKDAY,10,410,1101,AM_Offpeak
8,1012,WEEKDAY,11,604,1247,AM_Offpeak
9,1012,WEEKDAY,12,674,1323,AM_Offpeak


In [187]:
df.drop(columns=["TimePerHour"], inplace=True)
df.set_index(["BusStopCode","DayType","TimeCat"], inplace=True)
df = df.groupby(["BusStopCode","DayType","TimeCat"]).agg({"Monthly_TapIn":"mean","Monthly_TapOut":"mean"})
# Shows missing values since some Bus Stops don't have categories because they have no Tap In and Tap Out
# at that time category. Hence, we should impute them with zeros (since their mean Tap In and Tap out
# would also be zero)
print(df.isnull().sum())
display(df[df.isnull().any(axis=1)])

Monthly_TapIn     953
Monthly_TapOut    953
dtype: int64


Monthly_TapIn  Monthly_TapOut
BusStopCode DayType          TimeCat                                  
1112        WEEKDAY          PM_Peak               NaN             NaN
1311        WEEKDAY          PM_Peak               NaN             NaN
1319        WEEKDAY          PM_Peak               NaN             NaN
3241        WEEKDAY          PM_Offpeak            NaN             NaN
            WEEKENDS/HOLIDAY AM_Peak               NaN             NaN
...                                                ...             ...
98209       WEEKENDS/HOLIDAY PM_Offpeak            NaN             NaN
98219       WEEKENDS/HOLIDAY PM_Peak               NaN             NaN
                             PM_Offpeak            NaN             NaN
98229       WEEKENDS/HOLIDAY PM_Peak               NaN             NaN
                             PM_Offpeak            NaN             NaN

[953 rows x 2 columns]

In [188]:
df.fillna(0, inplace=True)
df.round()
df[['Monthly_TapIn','Monthly_TapOut']] = df[['Monthly_TapIn','Monthly_TapOut']].astype("int")
print(df.isnull().sum())
display(df)

Monthly_TapIn     0
Monthly_TapOut    0
dtype: int64


Monthly_TapIn  Monthly_TapOut
BusStopCode DayType          TimeCat                                  
1012        WEEKDAY          AM_Peak               323             592
                             AM_Offpeak            757            1189
                             PM_Peak              1442            1326
                             PM_Offpeak            609             325
            WEEKENDS/HOLIDAY AM_Peak               113             201
...                                                ...             ...
99189       WEEKDAY          PM_Offpeak             33             207
            WEEKENDS/HOLIDAY AM_Peak                10              45
                             AM_Offpeak             37              93
                             PM_Peak                60             125
                             PM_Offpeak             30              58

[40688 rows x 2 columns]

In [189]:
df.reset_index(inplace=True)

In [190]:
# Long to wide for TimeCat (pivot)
df = df.pivot(index=["BusStopCode","DayType"], columns="TimeCat", values=["Monthly_TapIn","Monthly_TapOut"])
df.columns = ["Monthly_TapIn_AM_Peak","Monthly_TapIn_AM_Offpeak","Monthly_TapIn_PM_Peak","Monthly_TapIn_PM_Offpeak","Monthly_TapOut_AM_Peak","Monthly_TapOut_AM_Offpeak","Monthly_TapOut_PM_Peak","Monthly_TapOut_PM_Offpeak"]
df.reset_index(inplace=True)

# Long to wide for DayType
df = df.pivot(index="BusStopCode", columns="DayType", values=["Monthly_TapIn_AM_Peak","Monthly_TapIn_AM_Offpeak","Monthly_TapIn_PM_Peak","Monthly_TapIn_PM_Offpeak","Monthly_TapOut_AM_Peak","Monthly_TapOut_AM_Offpeak","Monthly_TapOut_PM_Peak","Monthly_TapOut_PM_Offpeak"])
df.columns = ["Weekday_Monthly_TapIn_AM_Peak","Weekend/Hols_Monthly_TapIn_AM_Peak","Weekday_Monthly_TapIn_AM_Offpeak","Weekend/Hols_Monthly_TapIn_AM_Offpeak","Weekday_Monthly_TapIn_PM_Peak","Weekend/Hols_Monthly_TapIn_PM_Peak","Weekday_Monthly_TapIn_PM_Offpeak","Weekend/Hols_Monthly_TapIn_PM_Offpeak","Weekday_Monthly_TapOut_AM_Peak","Weekend/Hols_Monthly_TapOut_AM_Peak","Weekday_Monthly_TapOut_AM_Offpeak","Weekend/Hols_Monthly_TapOut_AM_Offpeak","Weekday_Monthly_TapOut_PM_Peak","Weekend/Hols_Monthly_TapOut_PM_Peak","Weekday_Monthly_TapOut_PM_Offpeak","Weekend/Hols_Monthly_TapOut_PM_Offpeak"]
display(df)

,Weekday_Monthly_TapIn_AM_Peak,Weekend/Hols_Monthly_TapIn_AM_Peak,Weekday_Monthly_TapIn_AM_Offpeak,Weekend/Hols_Monthly_TapIn_AM_Offpeak,Weekday_Monthly_TapIn_PM_Peak,Weekend/Hols_Monthly_TapIn_PM_Peak,Weekday_Monthly_TapIn_PM_Offpeak,Weekend/Hols_Monthly_TapIn_PM_Offpeak,Weekday_Monthly_TapOut_AM_Peak,Weekend/Hols_Monthly_TapOut_AM_Peak,Weekday_Monthly_TapOut_AM_Offpeak,Weekend/Hols_Monthly_TapOut_AM_Offpeak,Weekday_Monthly_TapOut_PM_Peak,Weekend/Hols_Monthly_TapOut_PM_Peak,Weekday_Monthly_TapOut_PM_Offpeak,Weekend/Hols_Monthly_TapOut_PM_Offpeak
BusStopCode,,,,,,,,,,,,,,,,
1012,323,113,757,375,1442,565,609,268,592,201,1189,634,1326,552,325,155
1013,181,61,515,291,1210,502,658,274,214,71,454,243,592,215,207,102
1019,235,70,571,274,678,313,198,69,742,241,976,497,728,337,198,74
1029,327,99,921,487,1498,647,463,198,703,196,785,409,635,270,140,59
1039,327,110,1354,866,1905,1221,1048,529,864,291,1714,1017,1642,911,446,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99139,0,0,2,0,0,1,0,0,535,241,569,451,642,540,305,187
99161,1,0,1,1,3,3,1,2,24,6,17,15,29,30,5,7
99171,4,2,11,9,16,16,6,5,40,35,83,90,78,151,18,18


In [191]:
df.to_csv('./data/bus_demand_low_LOD.csv')

## Dashboard 1
### Dataset 2 (Bus Capacity)
url: https://datamall.lta.gov.sg/content/datamall/en/dynamic-data.html

Used: Bus Services, Bus Stops, Bus Routes

In [192]:
# Convert json from a file into dictionary
stops = pd.read_csv('./datasrc/bus_stops.csv', na_values=['-','NaN'])
routes = pd.read_csv('./datasrc/bus_routes.csv', na_values=['-','NaN'])
services = pd.read_csv('./datasrc/bus_services.csv', na_values=['-','NaN'])
routes = routes.drop(columns = ['Unnamed: 0'])
services = services.drop(columns = ['Unnamed: 0'])
stops = stops.drop(columns = ['Unnamed: 0'])

In [193]:
# Apparently there is just one value in routes that is not numeric (E0323)
tmp = [x for x in routes["BusStopCode"] if not x.isnumeric()]
print(tmp)

['E0323']


In [194]:
# There is only one row with the BusStopCode E0323 and it is on row 18622
print(routes[routes['BusStopCode']=="E0323"])

      ServiceNo Operator  Direction  StopSequence BusStopCode  Distance  \
18622       700     SMRT          2             2       E0323       0.0   

       WD_FirstBus  WD_LastBus  SAT_FirstBus  SAT_LastBus SUN_FirstBus  \
18622        630.0      1930.0         630.0       1400.0          NaN   

       SUN_LastBus  
18622          NaN  


In [195]:
# Since the value with the faulty BusStopCode (E0323) has essentially the same data as the row before it, we can
# safely drop the row
display(routes.iloc[18621:18624])
routes.drop(18622, inplace=True)

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
18621,700,SMRT,2,1,03239,0.0,630.0,1930.0,630.0,1400.0,NaN,NaN
18622,700,SMRT,2,2,E0323,0.0,630.0,1930.0,630.0,1400.0,NaN,NaN
18623,700,SMRT,2,3,03223,0.7,632.0,1933.0,632.0,1403.0,NaN,NaN


In [196]:
# Now I can safely convert the row into an int
routes["BusStopCode"] = routes["BusStopCode"].astype(int)

In [197]:
df1 = pd.merge(routes, stops, on="BusStopCode", how="inner")
df1.sort_values(['ServiceNo', 'Direction','StopSequence'], inplace=True)
df = pd.merge(df1, services, on=["ServiceNo","Operator","Direction"], how="inner")
print(len(df))

26485


In [198]:
def avg_list(series_element):
    if type(series_element) == list:
        # If it has two values (eg. 9-12 converted into [9,12])
        if len(series_element)==2:
            return (float(series_element[0])+float(series_element[1]))//2
        else:
            # If it has one value (e.g. 29 converted into [29])
            return float(series_element[0])
    else:
        # If it is NaN (since the bus did not provide any information, it does not operate at that time and thus has
        # no passengers at that frequency)
        return 0

In [199]:
AM_PEAK = df['AM_Peak_Freq'].str.split('-')
AM_PEAK = AM_PEAK.apply(avg_list)
AM_OFFPEAK = df['AM_Offpeak_Freq'].str.split('-')
AM_OFFPEAK = AM_OFFPEAK.apply(avg_list)
PM_PEAK = df['PM_Peak_Freq'].str.split('-')
PM_PEAK = PM_PEAK.apply(avg_list)
PM_OFFPEAK = df['PM_Offpeak_Freq'].str.split('-')
PM_OFFPEAK = PM_OFFPEAK.apply(avg_list)

df['AM_Peak_avg'] = AM_PEAK
df['AM_Offpeak_avg'] = AM_OFFPEAK
df['PM_Peak_avg'] = PM_PEAK
df['PM_Offpeak_avg'] = PM_OFFPEAK

In [200]:
df.set_index(['ServiceNo','Direction','StopSequence'], inplace=True)
df.sort_index()
display(df)

Operator  BusStopCode  Distance  WD_FirstBus  \
ServiceNo Direction StopSequence                                                
10        1         1                SBST        75009       0.0        500.0   
                    2                SBST        76059       0.6        502.0   
                    3                SBST        76069       1.1        504.0   
                    4                SBST        96289       2.3        508.0   
                    5                SBST        96109       2.7        509.0   
...                                   ...          ...       ...          ...   
NR8       1         57               SMRT        43839      23.1          NaN   
                    58               SMRT        43649      23.4          NaN   
                    59               SMRT        43589      23.9          NaN   
                    60               SMRT        43579      24.3          NaN   
                    61               SMRT        43609      24.6          NaN   

                                  WD_LastBus  SAT_FirstBus  SAT_LastBus  \
ServiceNo Direction StopSequence                                          
10        1         1                 2300.0         500.0       2300.0   
                    2                 2302.0         502.0       2302.0   
                    3                 2304.0         504.0       2304.0   
                    4                 2308.0         508.0       2309.0   
                    5                 2310.0         509.0       2311.0   
...                                      ...           ...          ...   
NR8       1         57                   NaN          28.0        254.0   
                    58                   NaN          29.0        255.0   
                    59                   NaN          30.0        256.0   
                    60                   NaN          31.0        257.0   
                    61                   NaN          32.0        258.0   

                                 SUN_FirstBus  SUN_LastBus  \
ServiceNo Direction StopSequence                             
10        1         1                    0500       2300.0   
                    2                    0502       2302.0   
                    3                    0503       2304.0   
                    4                    0507       2308.0   
                    5                    0508       2309.0   
...                                       ...          ...   
NR8       1         57                    NaN          NaN   
                    58                    NaN          NaN   
                    59                    NaN          NaN   
                    60                    NaN          NaN   
                    61                    NaN          NaN   

                                             RoadName  ... DestinationCode  \
ServiceNo Direction StopSequence                       ...                   
10        1         1                 Tampines Ctrl 1  ...         16009.0   
                    2                  Tampines Ave 5  ...         16009.0   
                    3                  Tampines Ave 5  ...         16009.0   
                    4                       Simei Ave  ...         16009.0   
                    5                       Simei Ave  ...         16009.0   
...                                               ...  ...             ...   
NR8       1         57            Bt Batok West Ave 7  ...         43609.0   
                    58            Bt Batok West Ave 7  ...         43609.0   
                    59            Bt Batok West Ave 5  ...         43609.0   
                    60            Bt Batok West Ave 5  ...         43609.0   
                    61            Bt Batok West Ave 5  ...         43609.0   

                                  AM_Peak_Freq  AM_Offpeak_Freq PM_Peak_Freq  \
ServiceNo Direction StopSequence                                               
10        1         1

In [201]:
# Check missing values in distance
df[df["Distance"].isnull()]

Operator  BusStopCode  Distance  WD_FirstBus  \
ServiceNo Direction StopSequence                                                
185       1         1                SBST        22609       NaN        545.0   
          2         1                SBST        11379       NaN        600.0   
200       1         1                SBST        11379       NaN        600.0   
252       1         37               SBST        22009       NaN        630.0   
2N        1         1                SBST         2099       NaN          NaN   

                                  WD_LastBus  SAT_FirstBus  SAT_LastBus  \
ServiceNo Direction StopSequence                                          
185       1         1                 2315.0         545.0       2315.0   
          2         1                 2300.0         600.0       2300.0   
200       1         1                 2329.0         600.0       2335.0   
252       1         37                   2.0         631.0       2400.0   
2N        1         1                    NaN        2400.0        200.0   

                                 SUN_FirstBus  SUN_LastBus  \
ServiceNo Direction StopSequence                             
185       1         1                    0545       2315.0   
          2         1                    0600       2300.0   
200       1         1                    0600       2336.0   
252       1         37                   0630          1.0   
2N        1         1                     NaN          NaN   

                                            RoadName  ... DestinationCode  \
ServiceNo Direction StopSequence                      ...                   
185       1         1                    Soon Lee Rd  ...         11389.0   
          2         1             Nth Buona Vista Rd  ...         22199.0   
200       1         1             Nth Buona Vista Rd  ...         11389.0   
252       1         37            Jurong West Ctrl 3  ...         22009.0   
2N        1         1                   Raffles Blvd  ...         75109.0   

                                  AM_Peak_Freq  AM_Offpeak_Freq PM_Peak_Freq  \
ServiceNo Direction StopSequence                                               
185       1         1                    10-18            11-18        12-15   
          2         1                    10-17            10-15        10-20   
200       1         1                     9-10             9-30         9-10   
252       1         37                    8-10             7-18          7-9   
2N        1         1                      NaN              NaN          NaN   

                                  PM_Offpeak_Freq     LoopDesc AM_Peak_avg  \
ServiceNo Direction StopSequence                                             
185       1         1                       13-15          NaN        14.0   
          2         1                       14-16          NaN        13.0   
200       1         1                       14-27  Clementi Rd         9.0   
252       1         37                       8-20  Joo Koon Rd         9.0   
2N        1         1                       40-40          NaN         0.0   

                                 AM_Offpeak_avg PM_Peak_avg PM_Offpeak_avg  
ServiceNo Direction StopSequence                                            
185       1         1                      14.0        13.0           14.0  
          2         1                      12.0        15.0           15.0  
200       1         1                      19.0         9.0           20.0  
252       1         37                     12.0         8.0           14.0  
2N        1         1                       0.0         0.0           40.0  

[5 rows x 25 columns]

In [202]:
# Determining missing values using Deterministic Regression Imputation
model = linear_model.LinearRegression()
# Removes the last value which is nan in y_values
x_values = np.array(df.loc["252",1].index.to_list())[0:-1]
# Reshape according to docs
x_values = x_values.reshape(-1,1)
# Removes the last value which is nan
y_values = np.array(df.loc[("252",1),"Distance"].to_list())[0:-1]
model.fit(X = x_values, y = y_values)

prediction = model.predict([[np.array(df.loc["252",1].index.to_list())[-1]]])

In [203]:
df.loc[("252",1,37),"Distance"] = round(prediction[0],1)
display(df.loc[("252",1,37)])

,,,Operator,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,...,DestinationCode,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq,LoopDesc,AM_Peak_avg,AM_Offpeak_avg,PM_Peak_avg,PM_Offpeak_avg
ServiceNo,Direction,StopSequence,,,,,,,,,,,,,,,,,,,,,
252,1,37,SBST,22009,14.2,630.0,2.0,631.0,2400.0,0630,1.0,Jurong West Ctrl 3,...,22009.0,8-10,7-18,7-9,8-20,Joo Koon Rd,9.0,12.0,8.0,14.0


In [204]:
# Since they are all the first stop, we can just fill with 0
df["Distance"].fillna(0, inplace=True)

In [205]:
# Drop the freq since we extracted it to avg and drop the loop
df.drop(columns=["AM_Peak_Freq","AM_Offpeak_Freq","PM_Peak_Freq","PM_Offpeak_Freq"], inplace=True)

In [206]:
df["LoopDesc"] = df["LoopDesc"].apply(lambda x: 'N' if type(x)==float else 'Y')
display(df)

Operator  BusStopCode  Distance  WD_FirstBus  \
ServiceNo Direction StopSequence                                                
10        1         1                SBST        75009       0.0        500.0   
                    2                SBST        76059       0.6        502.0   
                    3                SBST        76069       1.1        504.0   
                    4                SBST        96289       2.3        508.0   
                    5                SBST        96109       2.7        509.0   
...                                   ...          ...       ...          ...   
NR8       1         57               SMRT        43839      23.1          NaN   
                    58               SMRT        43649      23.4          NaN   
                    59               SMRT        43589      23.9          NaN   
                    60               SMRT        43579      24.3          NaN   
                    61               SMRT        43609      24.6          NaN   

                                  WD_LastBus  SAT_FirstBus  SAT_LastBus  \
ServiceNo Direction StopSequence                                          
10        1         1                 2300.0         500.0       2300.0   
                    2                 2302.0         502.0       2302.0   
                    3                 2304.0         504.0       2304.0   
                    4                 2308.0         508.0       2309.0   
                    5                 2310.0         509.0       2311.0   
...                                      ...           ...          ...   
NR8       1         57                   NaN          28.0        254.0   
                    58                   NaN          29.0        255.0   
                    59                   NaN          30.0        256.0   
                    60                   NaN          31.0        257.0   
                    61                   NaN          32.0        258.0   

                                 SUN_FirstBus  SUN_LastBus  \
ServiceNo Direction StopSequence                             
10        1         1                    0500       2300.0   
                    2                    0502       2302.0   
                    3                    0503       2304.0   
                    4                    0507       2308.0   
                    5                    0508       2309.0   
...                                       ...          ...   
NR8       1         57                    NaN          NaN   
                    58                    NaN          NaN   
                    59                    NaN          NaN   
                    60                    NaN          NaN   
                    61                    NaN          NaN   

                                             RoadName  ...  Latitude  \
ServiceNo Direction StopSequence                       ...             
10        1         1                 Tampines Ctrl 1  ...  1.354076   
                    2                  Tampines Ave 5  ...  1.352962   
                    3                  Tampines Ave 5  ...  1.348753   
                    4                       Simei Ave  ...  1.340055   
                    5                       Simei Ave  ...  1.337371   
...                                               ...  ...       ...   
NR8       1         57            Bt Batok West Ave 7  ...  1.365273   
                    58            Bt Batok West Ave 7  ...  1.365458   
                    59            Bt Batok West Ave 5  ...  1.362273   
                    60            Bt Batok West Ave 5  ...  1.359117   
                    61            Bt Batok West Ave 5  ...  1.356536   

                                   Longitude     Category OriginCode  \
ServiceNo Direction StopSequence                                       
10        1         1             103.943391        TRUNK    75009.0   
                    2             103.94165

In [207]:
df.dtypes
# Drop temporarily
df.drop(columns=["WD_FirstBus","WD_LastBus","SAT_FirstBus","SAT_LastBus","SUN_FirstBus","SUN_LastBus"], inplace=True)

# Convert Origin and Destination Code to int
df[['OriginCode','DestinationCode']]=df[['OriginCode','DestinationCode']].astype(int)

# Convert freq avg to int
df[['AM_Peak_avg','AM_Offpeak_avg','PM_Peak_avg','PM_Offpeak_avg']]=df[['AM_Peak_avg','AM_Offpeak_avg','PM_Peak_avg','PM_Offpeak_avg']].astype(int)
display(df)


Operator  BusStopCode  Distance  \
ServiceNo Direction StopSequence                                   
10        1         1                SBST        75009       0.0   
                    2                SBST        76059       0.6   
                    3                SBST        76069       1.1   
                    4                SBST        96289       2.3   
                    5                SBST        96109       2.7   
...                                   ...          ...       ...   
NR8       1         57               SMRT        43839      23.1   
                    58               SMRT        43649      23.4   
                    59               SMRT        43589      23.9   
                    60               SMRT        43579      24.3   
                    61               SMRT        43609      24.6   

                                             RoadName           Description  \
ServiceNo Direction StopSequence                                              
10        1         1                 Tampines Ctrl 1          Tampines Int   
                    2                  Tampines Ave 5  Opp Our Tampines Hub   
                    3                  Tampines Ave 5               Blk 147   
                    4                       Simei Ave   Changi General Hosp   
                    5                       Simei Ave          Opp Blk 3012   
...                                               ...                   ...   
NR8       1         57            Bt Batok West Ave 7           Opp Blk 336   
                    58            Bt Batok West Ave 7            HomeTeamNS   
                    59            Bt Batok West Ave 5           Opp Blk 347   
                    60            Bt Batok West Ave 5         Bt Gombak Stn   
                    61            Bt Batok West Ave 5               Blk 531   

                                  Latitude   Longitude     Category  \
ServiceNo Direction StopSequence                                      
10        1         1             1.354076  103.943391        TRUNK   
                    2             1.352962  103.941652        TRUNK   
                    3             1.348753  103.942086        TRUNK   
                    4             1.340055  103.948381        TRUNK   
                    5             1.337371  103.950673        TRUNK   
...                                    ...         ...          ...   
NR8       1         57            1.365273  103.748250  NIGHT RIDER   
                    58            1.365458  103.750602  NIGHT RIDER   
                    59            1.362273  103.751532  NIGHT RIDER   
                    60            1.359117  103.751668  NIGHT RIDER   
                    61            1.356536  103.749924  NIGHT RIDER   

                                  OriginCode  DestinationCode LoopDesc  \
ServiceNo Direction StopSequence                                         
10        1         1                  75009            16009        N   
                    2                  75009            16009        N   
                    3                  75009            16009        N   
                    4                  75009            16009        N   
                    5                  75009            16009        N   
...                                      ...              ...      ...   
NR8       1         57                  2099            43609        Y   
                    58                  2099            43609        Y   
                    59                  2099            43609        Y   
                    60                  2099            43609        Y   
                    61                  2099            43609        Y   

                                  AM_Peak_avg  AM_Offpeak_avg  PM_Peak_avg  \
ServiceNo Direction StopSequence                                             
10        1         1                       8              11           12   

In [208]:
print(df.isnull().sum())
print(df.dtypes)

Operator           0
BusStopCode        0
Distance           0
RoadName           0
Description        0
Latitude           0
Longitude          0
Category           0
OriginCode         0
DestinationCode    0
LoopDesc           0
AM_Peak_avg        0
AM_Offpeak_avg     0
PM_Peak_avg        0
PM_Offpeak_avg     0
dtype: int64
Operator            object
BusStopCode          int32
Distance           float64
RoadName            object
Description         object
Latitude           float64
Longitude          float64
Category            object
OriginCode           int32
DestinationCode      int32
LoopDesc            object
AM_Peak_avg          int32
AM_Offpeak_avg       int32
PM_Peak_avg          int32
PM_Offpeak_avg       int32
dtype: object


In [209]:
# Write to csv
df.to_csv('data/bus_capacity.csv')

### Merge low LOD bus demand with bus capacity

In [210]:
df1 = pd.read_csv('data/bus_capacity.csv')
df2 = pd.read_csv('data/bus_demand_low_LOD.csv')

In [211]:
df = pd.merge(df1, df2, on="BusStopCode", how="left")

In [212]:
null = df[df.isnull().any(axis=1)]
display(null.sort_values("ServiceNo"))

,ServiceNo,Direction,StopSequence,Operator,BusStopCode,Distance,RoadName,Description,Latitude,Longitude,...,Weekday_Monthly_TapIn_PM_Offpeak,Weekend/Hols_Monthly_TapIn_PM_Offpeak,Weekday_Monthly_TapOut_AM_Peak,Weekend/Hols_Monthly_TapOut_AM_Peak,Weekday_Monthly_TapOut_AM_Offpeak,Weekend/Hols_Monthly_TapOut_AM_Offpeak,Weekday_Monthly_TapOut_PM_Peak,Weekend/Hols_Monthly_TapOut_PM_Peak,Weekday_Monthly_TapOut_PM_Offpeak,Weekend/Hols_Monthly_TapOut_PM_Offpeak
404,103,1,40,SBST,59671,17.5,Yishun Ave 2,Bef Yishun Stn,1.427776,103.835187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1738,122,1,18,SBST,11479,7.2,Margaret Dr,The Fisherman Ch,1.299381,103.803720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2429,130,2,26,SBST,51041,10.4,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2668,132,2,24,SBST,51041,10.5,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3531,141,1,21,SBST,51041,6.8,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26160,NR3,1,58,SMRT,45409,27.9,Choa Chu Kang Cres,Blk 675,1.401898,103.745796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26161,NR3,1,59,SMRT,45381,28.1,Choa Chu Kang Nth 7,Blk 675A (LP 25),1.400859,103.747128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26163,NR3,1,61,SMRT,45479,29.0,Choa Chu Kang Dr,Kranji Pr Sch,1.394295,103.746520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26236,NR5,1,76,SMRT,22571,29.6,Boon Lay Way,Opp Boon Lay Stn,1.337804,103.706891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
# Impute with the mean of tap in and tap out from the same service number and direction
null_subset = null[["ServiceNo","Direction"]].drop_duplicates()
print(len(null_subset))

tmp = df[["ServiceNo","Direction"]].stack().isin(null_subset.stack().values).unstack()

# print all values from df in tmp where at least 2 True
df[tmp.any(axis=1)]


67


,ServiceNo,Direction,StopSequence,Operator,BusStopCode,Distance,RoadName,Description,Latitude,Longitude,...,Weekday_Monthly_TapIn_PM_Offpeak,Weekend/Hols_Monthly_TapIn_PM_Offpeak,Weekday_Monthly_TapOut_AM_Peak,Weekend/Hols_Monthly_TapOut_AM_Peak,Weekday_Monthly_TapOut_AM_Offpeak,Weekend/Hols_Monthly_TapOut_AM_Offpeak,Weekday_Monthly_TapOut_PM_Peak,Weekend/Hols_Monthly_TapOut_PM_Peak,Weekday_Monthly_TapOut_PM_Offpeak,Weekend/Hols_Monthly_TapOut_PM_Offpeak
0,10,1,1,SBST,75009,0.0,Tampines Ctrl 1,Tampines Int,1.354076,103.943391,...,1088.0,633.0,0.0,0.0,0.0,0.0,0.0,0.0,374.0,448.0
1,10,1,2,SBST,76059,0.6,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652,...,1645.0,526.0,1621.0,369.0,1184.0,562.0,979.0,482.0,240.0,84.0
2,10,1,3,SBST,76069,1.1,Tampines Ave 5,Blk 147,1.348753,103.942086,...,1014.0,318.0,1838.0,563.0,1552.0,662.0,1828.0,652.0,788.0,224.0
3,10,1,4,SBST,96289,2.3,Simei Ave,Changi General Hosp,1.340055,103.948381,...,522.0,195.0,0.0,933.0,1653.0,624.0,1989.0,797.0,503.0,205.0
4,10,1,5,SBST,96109,2.7,Simei Ave,Opp Blk 3012,1.337371,103.950673,...,131.0,60.0,1641.0,341.0,728.0,226.0,620.0,196.0,181.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26480,NR8,1,57,SMRT,43839,23.1,Bt Batok West Ave 7,Opp Blk 336,1.365273,103.748250,...,78.0,49.0,370.0,99.0,340.0,160.0,419.0,150.0,170.0,83.0
26481,NR8,1,58,SMRT,43649,23.4,Bt Batok West Ave 7,HomeTeamNS,1.365458,103.750602,...,246.0,132.0,444.0,87.0,417.0,204.0,613.0,193.0,235.0,126.0
26482,NR8,1,59,SMRT,43589,23.9,Bt Batok West Ave 5,Opp Blk 347,1.362273,103.751532,...,166.0,68.0,279.0,87.0,298.0,160.0,392.0,158.0,114.0,52.0
26483,NR8,1,60,SMRT,43579,24.3,Bt Batok West Ave 5,Bt Gombak Stn,1.359117,103.751668,...,1450.0,518.0,0.0,1925.0,0.0,1919.0,0.0,1480.0,899.0,270.0


In [214]:
missing_values = df[df.isnull().any(axis=1)]
similar_routes = df[df['ServiceNo'].isin(missing_values['ServiceNo']) & df['Direction'].isin(missing_values['Direction'])]
display(similar_routes)

,ServiceNo,Direction,StopSequence,Operator,BusStopCode,Distance,RoadName,Description,Latitude,Longitude,...,Weekday_Monthly_TapIn_PM_Offpeak,Weekend/Hols_Monthly_TapIn_PM_Offpeak,Weekday_Monthly_TapOut_AM_Peak,Weekend/Hols_Monthly_TapOut_AM_Peak,Weekday_Monthly_TapOut_AM_Offpeak,Weekend/Hols_Monthly_TapOut_AM_Offpeak,Weekday_Monthly_TapOut_PM_Peak,Weekend/Hols_Monthly_TapOut_PM_Peak,Weekday_Monthly_TapOut_PM_Offpeak,Weekend/Hols_Monthly_TapOut_PM_Offpeak
365,103,1,1,SBST,66009,0.0,S'goon Ave 2,S'Goon Int,1.350466,103.871690,...,967.0,1503.0,2168.0,979.0,0.0,2276.0,0.0,0.0,324.0,357.0
366,103,1,2,SBST,62131,0.5,Upp S'goon Rd,S'Goon Stn Exit H,1.348979,103.872774,...,1864.0,809.0,1564.0,427.0,1919.0,1008.0,0.0,1145.0,704.0,302.0
367,103,1,3,SBST,62141,0.9,Upp S'goon Rd,Opp Blk 1,1.351491,103.875152,...,372.0,172.0,467.0,181.0,418.0,221.0,563.0,186.0,220.0,90.0
368,103,1,4,SBST,63141,1.3,Boundary Rd,Blk 209,1.354046,103.874913,...,76.0,38.0,141.0,87.0,175.0,101.0,351.0,99.0,92.0,46.0
369,103,1,5,SBST,63361,1.8,Yio Chu Kang Link,Opp S'Goon Sports Cplx,1.357446,103.873840,...,281.0,144.0,963.0,272.0,1340.0,674.0,2432.0,844.0,949.0,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26321,NR6,1,83,SMRT,67399,35.0,Sengkang East Way,Blk 325 CP,1.393850,103.891670,...,321.0,111.0,712.0,107.0,709.0,306.0,1355.0,348.0,530.0,203.0
26322,NR6,1,84,SMRT,67581,35.4,Anchorvale Link,Blk 319A,1.391042,103.889747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26323,NR6,1,85,SMRT,67351,35.7,Anchorvale Dr,Blk 321 CP,1.389952,103.891344,...,21.0,8.0,203.0,28.0,266.0,117.0,498.0,140.0,201.0,100.0
26324,NR6,1,86,SMRT,67341,36.2,Compassvale Dr,Opp Blk 204 CP,1.388648,103.895052,...,52.0,24.0,367.0,59.0,292.0,139.0,519.0,176.0,184.0,88.0


In [215]:
# Get the mean of the tap in and tap out for the similar routes
similar_routes_mean = similar_routes.groupby(['ServiceNo','Direction']).mean().round()

# Assign the values back to missing values
df.set_index(['ServiceNo','Direction'], inplace=True)
df.fillna(similar_routes_mean, inplace=True)
df.reset_index(inplace=True)
display(df.loc[np.array(missing_values.index)])

,ServiceNo,Direction,StopSequence,Operator,BusStopCode,Distance,RoadName,Description,Latitude,Longitude,...,Weekday_Monthly_TapIn_PM_Offpeak,Weekend/Hols_Monthly_TapIn_PM_Offpeak,Weekday_Monthly_TapOut_AM_Peak,Weekend/Hols_Monthly_TapOut_AM_Peak,Weekday_Monthly_TapOut_AM_Offpeak,Weekend/Hols_Monthly_TapOut_AM_Offpeak,Weekday_Monthly_TapOut_PM_Peak,Weekend/Hols_Monthly_TapOut_PM_Peak,Weekday_Monthly_TapOut_PM_Offpeak,Weekend/Hols_Monthly_TapOut_PM_Offpeak
404,103,1,40,SBST,59671,17.5,Yishun Ave 2,Bef Yishun Stn,1.427776,103.835187,...,299.0,151.0,586.0,218.0,429.0,257.0,695.0,261.0,356.0,150.0
1738,122,1,18,SBST,11479,7.2,Margaret Dr,The Fisherman Ch,1.299381,103.803720,...,291.0,183.0,585.0,274.0,559.0,357.0,620.0,343.0,310.0,183.0
2429,130,2,26,SBST,51041,10.4,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,392.0,233.0,507.0,160.0,521.0,317.0,531.0,383.0,288.0,191.0
2668,132,2,24,SBST,51041,10.5,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,407.0,227.0,623.0,314.0,632.0,406.0,798.0,426.0,398.0,185.0
3531,141,1,21,SBST,51041,6.8,Thomson Rd,Opp SLF Cplx,1.333436,103.837917,...,339.0,177.0,523.0,143.0,526.0,261.0,747.0,311.0,316.0,156.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26160,NR3,1,58,SMRT,45409,27.9,Choa Chu Kang Cres,Blk 675,1.401898,103.745796,...,529.0,292.0,448.0,210.0,704.0,409.0,735.0,478.0,386.0,222.0
26161,NR3,1,59,SMRT,45381,28.1,Choa Chu Kang Nth 7,Blk 675A (LP 25),1.400859,103.747128,...,529.0,292.0,448.0,210.0,704.0,409.0,735.0,478.0,386.0,222.0
26163,NR3,1,61,SMRT,45479,29.0,Choa Chu Kang Dr,Kranji Pr Sch,1.394295,103.746520,...,529.0,292.0,448.0,210.0,704.0,409.0,735.0,478.0,386.0,222.0
26236,NR5,1,76,SMRT,22571,29.6,Boon Lay Way,Opp Boon Lay Stn,1.337804,103.706891,...,441.0,256.0,737.0,292.0,786.0,509.0,824.0,557.0,488.0,251.0


In [216]:
# Write back
df.to_csv('data/bus_demand_and_capacity.csv', index=False)